In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re

from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 경고표시 지우기
import warnings
warnings.filterwarnings(action="ignore")

In [3]:
# 0 1 2 3 4 5 => 몇 퍼센트로 긍정적인 가게입니다?
# 스코어가 나왔을 때, 콘텐츠 기반 필터링은?? => 콘텐츠 기반 필터링을 하려면 유샤도를 측정해야함. 즉, 메뉴를 정해야한다는 의미인듯?
# 한식 / 일식 / 중식 / 양식등 메뉴를 정함(정하는 거 부터가;;) -> db를 쫙 펴서 유사도 검정 -> 유사한 곳을 스코어 순으로 정렬 -> 상위 5개 추천?
# 처음에는 상위 5개 추천 -> 한 곳을 골라서 간다면 그 기록이 남고 그 다음부터 해당 가게와 유사한 곳과 상위 5군데를 모두 추천
# 또 그 다음에는 그 전에 선택한 곳과 그 전전번에 선택한 곳을 기반으로 유사한 곳을 추천 -> 이렇게 가지치기????

# max_score = 5

# score = [0,1,2,3,5,5]

# score = pd.DataFrame(score, columns=["review_predict"])

# model_load
filepath = "./data/model/model_lstm_220110.h5"

model = load_model(filepath, custom_objects=None, compile=True)

In [10]:
# for data in score["review_predict"]:
#     score_ = (data*100) / 5
#     print("이 가게는 {}%로 긍정적인 반응을 얻고 있습니다".format(score_))

이 가게는 0.0%로 긍정적인 반응을 얻고 있습니다
이 가게는 20.0%로 긍정적인 반응을 얻고 있습니다
이 가게는 40.0%로 긍정적인 반응을 얻고 있습니다
이 가게는 60.0%로 긍정적인 반응을 얻고 있습니다
이 가게는 100.0%로 긍정적인 반응을 얻고 있습니다
이 가게는 100.0%로 긍정적인 반응을 얻고 있습니다


In [81]:
mango = pd.read_csv("./data/리뷰_6개구 수정_0112.csv")

In [82]:
mango

,Unnamed: 0,이름,평점,주소,주력 메뉴,가격대,망고플레이트 주소,리뷰내용,구
0,0,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,가로수길에 있는 오랜 참치맛집 e-청우참치위치를 옮기다!!원래 있...,강남구
1,1,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,"가성비 너무 좋고, 정말 양질의 참치를 양껏 즐길 수 있어서 더 ...",강남구
2,2,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,가성비 좋은 한국식 참치횟집. 선도가 꽤 좋다. 카운터석도 있고 ...,강남구
3,3,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,그저 박수만. 참치땡길때 여기만한 곳이 없다.,강남구
4,4,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,맛있다.. 느끼해서 많이 못먹는게 함정.,강남구
...,...,...,...,...,...,...,...,...,...
44415,44415,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,성북동에서 까페를 찾다가 가게되었어요. 구조가 뭔가 특이하고 분위...,성북구
44416,44416,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,* 한줄평 : 아지트 삼고싶은 성북동 골목 카페성북동 언덕 아래 ...,성북구
44417,44417,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,조용한 성북동에있는 가격도 저렴하고 부니기 좋은 카페 ㅎㅎ 사람들...,성북구
44418,44418,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,리뷰 썼다가 사람 많아질까 두렵지만... 그렇지만 또 장사가 잘 ...,성북구


In [84]:
# 클러스터링된 가게 리스트
store_list = ["청우참치", "키미노돼지국밥", "조개나루터"] # 등등 임의로 뽑아냄

# # 먼저 평점을 매겨야하니까 일단 모델 돌려서 평점부터 써준다.
store_predictScore = []

for store in store_list:
    print(mango[mango["이름"]==store])
    
    score_sum = 0
    cnt=0

    for review in mango[mango["이름"]==store]["리뷰내용"]:
        test_sentence = review
        test_sentence = test_sentence.split(' ')
        test_sentences = []
        now_sentence = []
        for word in test_sentence:
            now_sentence.append(word)
            test_sentences.append(now_sentence[:])

        tokenizer = Tokenizer(num_words=3000)
        
        test_X_1 = tokenizer.texts_to_sequences(test_sentences)
        test_X_1 = pad_sequences(test_X_1, padding='post', maxlen=30)
        prediction = model.predict(test_X_1)
        for idx, sentence in enumerate(test_sentences):
            review_score = np.argmax((prediction[idx]).round(3))
        print("리뷰\n", review)
        print("실제 점수 : ", mango[mango["이름"]==store].iloc[cnt,2])
        print("예측한 점수 : ", review_score)
        print("-"*70)
        score_sum += review_score
        cnt+=1
    print("예측한 평균 평점 : {}".format((score_sum/cnt).round(2)))
    store_predictScore.append([store, (score_sum/cnt).round(2)])
    
print(store_predictScore)
# for data in store_index:
#     print(data[0])

    Unnamed: 0    이름   평점                  주소   주력 메뉴     가격대  \
0            0  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
1            1  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
2            2  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
3            3  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
4            4  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
5            5  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
6            6  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
7            7  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
8            8  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
9            9  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
10          10  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
11          11  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
12          12  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
13          13  청우참치  4.5  서울특별시 강남구 강남대로 632  회 / 스시  4만원 이상   
14          14  청우참치  4.5

In [86]:
# 가게 평점 정규표현식

# score_na = mango[mango["평점"]=="-"]["평점"].map({"-" : 0})

# score_na

# tmp_review = []

# for review_score in mango["평점"]:
#     if review_score == "-":
# #         pass
#         tmp_review.append(0)
#     else:
#         score_ = re.sub(r'[^0-9.0-9]', '', review_score)
# #         print(score_)
#         tmp_review.append(score_)
    
# tmp_review = np.array(tmp_review).astype(np.float32)
# tmp_review

# mango["review_score"] = tmp_review

# 그냥 가게별로 예측 리뷰 컬럼을 만들어야댐
store_list = list(mango["이름"].unique()) # 등등 임의로 뽑아냄


# 먼저 평점을 매겨야하니까 일단 모델 돌려서 평점부터 써준다.
store_predictScore = []

for store in store_list:
#     print(mango[mango["이름"]==store])
    
    score_sum = 0
    cnt=0

    for review in mango[mango["이름"]==store]["리뷰내용"]:
        test_sentence = review
        test_sentence = test_sentence.split(' ')
        test_sentences = []
        now_sentence = []
        for word in test_sentence:
            now_sentence.append(word)
            test_sentences.append(now_sentence[:])

        tokenizer = Tokenizer(num_words=3000)
        
        test_X_1 = tokenizer.texts_to_sequences(test_sentences)
        test_X_1 = pad_sequences(test_X_1, padding='post', maxlen=30)
        prediction = model.predict(test_X_1)
        for idx, sentence in enumerate(test_sentences):
            review_score = np.argmax((prediction[idx]).round(3))
#         print("리뷰\n", review)
#         print("실제 점수 : ", mango[mango["이름"]==store].iloc[cnt,2])
#         print("예측한 점수 : ", review_score)
#         print("-"*70)
        score_sum += review_score
        cnt+=1
#     print("예측한 평균 평점 : {}".format((score_sum/cnt).round(2)))
    store_predictScore.append([store, (score_sum/cnt).round(2)])
    
print(store_predictScore)
# for data in store_index:
#     print(data[0])

[['청우참치', 5.0], ['정돈', 5.0], ['꼼다비뛰드', 5.0], ['노들강', 5.0], ['영동소금구이', 5.0], ['야키토리묵', 5.0], ['목포집', 5.0], ['야사이마끼 쿠이신보', 5.0], ['칙피스', 5.0], ['콴안다오', 5.0], ['삼창교자', 5.0], ['신비갈비살', 5.0], ['정돈 프리미엄', 5.0], ['테일러커피', 5.0], ['오사카카레콘유', 5.0], ['레호이', 5.0], ['신사치킨클럽', 5.0], ['양인환대', 5.0], ['이디야커피랩', 5.0], ['고급오복수산', 5.0], ['치즈룸', 5.0], ['홍명', 5.0], ['모노로그', 5.0], ['탄광', 5.0], ['미라이', 5.0], ['일무레또', 5.0], ['비스트로앤트로', 5.0], ['상석', 5.0], ['압구정공주떡집', 5.0], ['야스노야지로', 5.0], ['비플레이트 바이 브라운브레드', 5.0], ['양재초밥', 5.0], ['원스타 올드패션드 햄버거', 5.0], ['DOMA ', 5.0], ['차만다', 5.0], ['이도', 5.0], ['지아니스나폴리', 5.0], ['양궈푸마라탕', 5.0], ['파티세리바이가루하루 ', 5.0], ['이도곰탕', 5.0], ['에빗', 5.0], ['한강수', 5.0], ['껠끄쇼즈', 5.0], ['땅코참숯구이', 5.0], ['왕스덕', 5.0], ['김수사', 5.0], ['은화계', 5.0], ['아이뽀유', 5.0], ['가온', 5.0], ['스시코우지', 5.0], ['매덕스피자', 5.0], ['효계', 5.0], ['길버트버거앤프라이즈', 5.0], ['와라야키 쿠이신보', 5.0], ['하쿠시', 5.0], ['새들러하우스', 5.0], ['스쿠로', 5.0], ['SOOT', 5.0], ['스시타츠', 5.0], ['볼피노리스토란테', 5.0], ['가디록', 5.0], ['스시선수', 5.0], ['갓포아키', 5.0]

In [87]:
# 망고플레이트 리뷰 데이터 빼고 
# 가게마다의 데이터로 만들고
# 예측 평점 컬럼 추가
# 코사인 유사도

# 가게마다 리뷰 빼기
# del mango["리뷰내용"]

# 망고 플레이트 데이터에 예측 평점 넣기
predict_score_list = []

for store in store_predictScore:
    predict_score = store[1]
    predict_score_list.append(predict_score)
    
# 망고 플레이트 가게별로 정리
# mango["주력 메뉴"].groupby(mango["이름"])
mango_df = mango.drop_duplicates(['이름'])

mango_df["predict_score"] = predict_score_list

mango_df

,Unnamed: 0,이름,평점,주소,주력 메뉴,가격대,망고플레이트 주소,리뷰내용,구,predict_score
0,0,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,가로수길에 있는 오랜 참치맛집 e-청우참치위치를 옮기다!!원래 있...,강남구,5.0
15,15,정돈,4.3,서울특별시 강남구 강남대로110길 26 1F,까스 요리,만원-2만원,https://www.mangoplate.com//restaurants/w_Uk02...,동그란 게 안심. 길쭉한 건 등심. 보이는 것보다 크기가 커서 다...,강남구,5.0
29,29,꼼다비뛰드,4.5,서울특별시 강남구 강남대로110길 62,베이커리,만원 미만,https://www.mangoplate.com//restaurants/DM2xUa...,재오픈한 꼼다에 부지런히 다녀왔어요! 오전 9시에 도착했더니 앞...,강남구,5.0
34,34,노들강,4.4,서울특별시 강남구 강남대로114길 19,해산물 요리,4만원 이상,https://www.mangoplate.com//restaurants/R7gZYp...,가격은 다소 있는편. 민어가 전문이나 민어 이외의 해산물도 있음....,강남구,5.0
41,41,영동소금구이,4.4,서울특별시 강남구 강남대로124길 10 1F,고기 요리,만원-2만원,https://www.mangoplate.com//restaurants/56P5Rz...,모든 메뉴가 딱히 흠 잡을 곳 없이 맛있었다. 모듬구이는 딱히 ...,강남구,5.0
...,...,...,...,...,...,...,...,...,...,...
44397,44397,소마,최신 버전 앱,서울특별시 성북구 성북로 89-1 1F,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/W_I7pp...,성북동에서 신앙생활을 하고 있는 신부님이 추천해주신 카페. 이 동...,성북구,5.0
44399,44399,동병상련,4.7 ...,서울특별시 성북구 정릉로10길 42-6,카페 / 디저트,만원-2만원,https://www.mangoplate.com//restaurants/FEPbvD...,"직접 핸드드립으로 내려주는 커피와,맛있는 떡들. 그리고 좋은 공간...",성북구,5.0
44403,44403,카페잇,최신 버전 앱,서울특별시 성북구 장월로1길 66,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/5p8t64...,여사장님 좀이상함;;맛은 그냥 무난한데여기 갔다가 기분 개나빴음ㅡ...,성북구,5.0
44406,44406,숙수도가,최신 버전 앱,서울특별시 성북구 동소문로17길 28,고기 요리,3만원-4만원,https://www.mangoplate.com//restaurants/iTWG1W...,너무 맛있어서 쓰러진 곳.오랜만에 서울와서 예약해놓은 고깃집에 갔...,성북구,5.0


In [88]:
# 여기서부터 추천 알고리즘 시작
mango_df[["주력 메뉴", "가격대"]]

mango_df[["주력 메뉴", "가격대"]].isna().sum()

mango_ = mango_df[["주력 메뉴", "가격대"]].fillna("0")

In [89]:
menu_list = []

for menu in mango_["주력 메뉴"]:
    menu = re.sub("\n", "",menu)
    menu_list.append(menu)
    
menu_list

mango_df["main_dish"] = np.array(menu_list)

price_list = []

for price in mango_["가격대"]:
    price = re.sub("\n", "",price)
    price_list.append(price)
    
price_list

mango_df["price"] = np.array(price_list)

In [90]:
# 이제 전처리가 안된 메뉴와 가격대를 삭제
mango_df

del mango_df["Unnamed: 0"]
del mango_df["주력 메뉴"]
del mango_df["가격대"]

In [94]:
mango_df.to_csv("./data/mango_recom_data.csv", index=False)
mango_df = pd.read_csv("./data/mango_recom_data.csv")

mango_df

,이름,평점,주소,망고플레이트 주소,리뷰내용,구,predict_score,main_dish,price
0,청우참치,4.5,서울특별시 강남구 강남대로 632,https://www.mangoplate.com//restaurants/YyRGx0...,가로수길에 있는 오랜 참치맛집 e-청우참치위치를 옮기다!!원래 있...,강남구,5.0,회 / 스시,4만원 이상
1,정돈,4.3,서울특별시 강남구 강남대로110길 26 1F,https://www.mangoplate.com//restaurants/w_Uk02...,동그란 게 안심. 길쭉한 건 등심. 보이는 것보다 크기가 커서 다...,강남구,5.0,까스 요리,만원-2만원
2,꼼다비뛰드,4.5,서울특별시 강남구 강남대로110길 62,https://www.mangoplate.com//restaurants/DM2xUa...,재오픈한 꼼다에 부지런히 다녀왔어요! 오전 9시에 도착했더니 앞...,강남구,5.0,베이커리,만원 미만
3,노들강,4.4,서울특별시 강남구 강남대로114길 19,https://www.mangoplate.com//restaurants/R7gZYp...,가격은 다소 있는편. 민어가 전문이나 민어 이외의 해산물도 있음....,강남구,5.0,해산물 요리,4만원 이상
4,영동소금구이,4.4,서울특별시 강남구 강남대로124길 10 1F,https://www.mangoplate.com//restaurants/56P5Rz...,모든 메뉴가 딱히 흠 잡을 곳 없이 맛있었다. 모듬구이는 딱히 ...,강남구,5.0,고기 요리,만원-2만원
...,...,...,...,...,...,...,...,...,...
4503,소마,최신 버전 앱,서울특별시 성북구 성북로 89-1 1F,https://www.mangoplate.com//restaurants/W_I7pp...,성북동에서 신앙생활을 하고 있는 신부님이 추천해주신 카페. 이 동...,성북구,5.0,카페 / 디저트,만원 미만
4504,동병상련,4.7 ...,서울특별시 성북구 정릉로10길 42-6,https://www.mangoplate.com//restaurants/FEPbvD...,"직접 핸드드립으로 내려주는 커피와,맛있는 떡들. 그리고 좋은 공간...",성북구,5.0,카페 / 디저트,만원-2만원
4505,카페잇,최신 버전 앱,서울특별시 성북구 장월로1길 66,https://www.mangoplate.com//restaurants/5p8t64...,여사장님 좀이상함;;맛은 그냥 무난한데여기 갔다가 기분 개나빴음ㅡ...,성북구,5.0,카페 / 디저트,만원 미만
4506,숙수도가,최신 버전 앱,서울특별시 성북구 동소문로17길 28,https://www.mangoplate.com//restaurants/iTWG1W...,너무 맛있어서 쓰러진 곳.오랜만에 서울와서 예약해놓은 고깃집에 갔...,성북구,5.0,고기 요리,3만원-4만원


In [95]:
###### 추천 알고리즘 ######
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
menu_mat = count_vect.fit_transform(mango_df["main_dish"])

print(menu_mat.shape)

(4508, 118)


In [96]:
from sklearn.metrics.pairwise import cosine_similarity

menu_sim = cosine_similarity(menu_mat, menu_mat)
menu_sim[0]

menu_c_sim = cosine_similarity(menu_mat,menu_mat).argsort()[:,::-1]

menu_c_sim.shape

def get_recommend_menu_list(df, store_title, top=30):
    # 특정 가게와 비슷한 가게를 추천해야 하기 때문에 "특정 가게" 정보를 뽑아냄
    target_store_index = df[df["이름"] == store_title].index.values
    print(target_store_index)
    
    # 코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = menu_c_sim[target_store_index, :top].reshape(-1)
    sim_index = sim_index[sim_index != target_store_index]
    print(sim_index)
    
    # data frame으로 만들고 vote_count로 정렬한 뒤 return
#     result = df.iloc[sim_index].sort_values("predict_score", ascending=False)[:5]
    result = df.iloc[sim_index]
    return result


In [98]:
result = get_recommend_menu_list(mango_df, store_title = "청우참치")

result.sort_values("평점", ascending=False)[:5]

[0]
[2966 1733  808 1028  727 3555 1551 2360  520 2894 3166 3551  516 1743
  298 4331  736 1288 4198  428 1010 3970 1007 4202 3547  802 3177 3559
  505]


,이름,평점,주소,망고플레이트 주소,리뷰내용,구,predict_score,main_dish,price
4331,우정초밥,4.6,서울특별시 성북구 종암로3길 31 1F,https://www.mangoplate.com//restaurants/jFWl95...,- 음주를 적극적으로 권장하는 다찌 스타일의 가성비 오마카세공식 ...,성북구,5.0,회 / 스시,2만원-3만원
2894,스시쇼부,4.4,서울특별시 은평구 은평로13길 11-9 2F,https://www.mangoplate.com//restaurants/mbVBPp...,"* 한줄평 : 가성비를 넘어선 갓성비, 런치 오마카세 38천원1....",은평구,5.0,회 / 스시,2만원-3만원
3166,스담,4.4,서울특별시 중구 다산로28길 2-1,https://www.mangoplate.com//restaurants/NaNdBa...,가성비 정말 좋게 먹은 스담! 런치 때 방문하면 디너 메뉴를 2...,중구,5.0,회 / 스시,만원-2만원
2966,스시쿠니,4.3,서울특별시 종로구 경교장길 35 팰리스에비뉴 1F 3133호,https://www.mangoplate.com//restaurants/sEa4_A...,주말 저녁 방문. 웨이팅 조금 있음. 주문메뉴: 2인세트(일반초...,종로구,5.0,회 / 스시,만원-2만원
1028,스시카야논데,4.3,서울특별시 동대문구 장한로27길 46 1F,https://www.mangoplate.com//restaurants/CU4Kmd...,"사실 여러 다른 분들처럼 스시에 조예가 깊진 않지만, 맛있게 먹...",동대문,5.0,회 / 스시,3만원-4만원
